# Section 1 — Report Header & Hypothesis

**Report Title:**  
Alt Text Versus No Alt Text on Engagement Rates in Bluesky Image Posts  

**Your Name:**  
Aubrey Nighman  

**Date:**  
October 5, 2025

---

### Hypothesis  
Posts on Bluesky that have **alt text for images** get a **greater average number of likes and reposts** than images and posts that do not have alt text.  

---

### Theoretical Rationale  
Features like alt text are made to make online activities more accessible and easy for users with disabilities. Research in media accessibility (Ellis & Kent, *Disability and New Media*, 2011) has proved that places that emphasize inclusiveness often experience better engagement, attention, and trust.

To further describe my theory, users who write good alt text and are more creative and take time with their postings, have increased the quality of the content which raises activity. Also, creators who make it a point to include accessability usually have and draw in followers who appreciate these things, which could raise activity and engagement, like likes, reposts, shares, etc.  

---

### Statistical Application  
In order to test this hypothesis, I will compare activity levels between posts that include alt text and ones that don't.  

- **Independent Variable:** Presence of alt text (`record.embed.images.alt`)  
- **Dependent Variables:** `likeCount`, `repostCount`  
- **Control Variables (optional):** Author’s `followersCount`, `postsCount`  

A **two-sample t-test** can show whether engagement differs between groups of alt text versus those who don't. 


# Section 2 — Endpoint Plan (Design Your Data Collection)

### Planned Endpoints

1. **`app.bsky.feed.searchPosts`** — to collect posts containing images.  
   - **Request parameters:**  
     - `q`: search for image-related keywords such as “photo,” “art,” or “illustration.”  
     - `limit`: number of posts to retrieve (e.g., 100–200 per request).  
   - **Response fields:**  
     - `post.uri`, `post.cid`, `record.embed.images.alt`, `likeCount`, `repostCount`, and `author.did`.  
   - **Why these fields map to the variables in the hypothesis:**  
     - Gives post metadata and image data, including if alt text is used.  

2. **`app.bsky.actor.getProfiles`** — to develop author data.  
   - **Request parameters:**  
     - `actor`: DID (identifier from post data).  
   - **Response fields:**  
     - `followersCount`, `followsCount`, `postsCount`, `createdAt`, `displayName`.  
   - **Why these fields map to the variables in the hypothesis:**  
     - Gives context for making engagement normal.

---

### Reliability and Bias  
- **Reliability:** Bluesky’s API provides structured JSON responses with consistency in the fields.  
- **Unreliability:** Some posts give out `record.embed.images`; rate limits restrict the larger samples.  
- **Bot Activity:** Bot/spam accounts can slightly adjust/change engagement averages.  
- **Ethics:** Only public data will be collected because of Bluesky’s API terms.  

---

### Limitations  
- Missing image objects lessen sample size.  
- Activity counts can slow down updates.  
- Some profiles have restricted data areas.  
- Alt text options and add ons does not always mean better quality, but can show more effort.


## Section 3 - Data Collection

In [3]:
# Imports
import requests      
import time           
import json as js    
import pandas as pd   

# Define URL for Bluesky API requests
BASE_URL = "https://api.bsky.app/xrpc"

## Data Collection (Endpoint 1):

In [4]:
#endpoint = f"{BASE_URL}/app.bsky.feed.searchPosts"
endpoint = f"{BASE_URL}/app.bsky.feed.searchPosts"
headers = {"User-Agent": "EMAT-Teaching/1.0 (+contact@example.com)"}
params = {"q": "photo", "limit": 50}

resp = requests.get(endpoint, params=params, headers=headers, timeout=30)

print("Status:", resp.status_code)
data = resp.json()
print("top-level keys:", list(data.keys()))
posts = data.get("posts", [])
rows = []

# Loop 
for p in posts:
    record = p.get("record", {})      
    embed = record.get("embed", {})    
    
    # Check if the embed contains any image that includes alt text
    images = embed.get("images", []) if isinstance(embed.get("images"), list) else []
    has_alt = any(img.get("alt") for img in images) if images else False

    stats = {
        "post_uri": p.get("uri"),
        "post_cid": p.get("cid"),
        "author_did": p.get("author", {}).get("did"),
        "likeCount": p.get("likeCount"),
        "repostCount": p.get("repostCount"),
        "has_alt_text": has_alt
    }
    ## Flatten the posts
#print(posts)
    rows.append(stats)

# Change the list into pandas DataFrame
posts_df = pd.DataFrame(rows)

# Display the first few rows to verify that the data loaded correctly
posts_df.head()


Status: 200
top-level keys: ['posts', 'cursor']


,post_uri,post_cid,author_did,likeCount,repostCount,has_alt_text
0,at://did:plc:thwspoidoksdkjdjclykbkzn/app.bsky...,bafyreifsmqoim5dymfbl6kdcimwk7etp57atbo7a247xb...,did:plc:thwspoidoksdkjdjclykbkzn,0,0,False
1,at://did:plc:5pzticukc5rgbmc7o3bso7il/app.bsky...,bafyreigcbegn5bciukerck5cjeeu6qr4fexcevkdrbuzc...,did:plc:5pzticukc5rgbmc7o3bso7il,0,0,True
2,at://did:plc:qixf4a5ossg2fkd3ztb3asyk/app.bsky...,bafyreie3u4smd47g5fsvyxryv66uxdj4utxqparferhkq...,did:plc:qixf4a5ossg2fkd3ztb3asyk,0,0,False
3,at://did:plc:ijk2uijbwllxr2z7q3yyzily/app.bsky...,bafyreic2fetoiwmtyi3q4qzcyqbuapkt5h2yau2m6v7tw...,did:plc:ijk2uijbwllxr2z7q3yyzily,0,0,False
4,at://did:plc:kjsl6b5t5klf5cbtnkyxq3ar/app.bsky...,bafyreibs4dtrb7slqvgghrtja6f7wwpostwoekafg6eo4...,did:plc:kjsl6b5t5klf5cbtnkyxq3ar,2,0,False


# Section 3 — Data Collection (Endpoint 2)

In [5]:

## Let us get profile data for all the authors from the previous feed
# get unique author ids which is dids
unique_dids = posts_df["author_did"].dropna().unique().tolist()
print("Number of unique authors:", len(unique_dids))
#print(unique_dids)

# Get author profiles for these dids
all_profiles = []

# Loop through each DID and collect profile info
for d in unique_dids:
    #print(js.dumps(d, indent=2))
    params = [("actor", d)]
    #print(d)
    resp = requests.get(f"{BASE_URL}/app.bsky.actor.getProfile", params=params, timeout=30)

    if resp.status_code != 200:
        print("Skipping", d, "status:", resp.status_code)
        continue

    data = resp.json()
    #print(js.dumps(data, indent=2))
    
    # Append this profile in our list 
    # flatten tha data for profile
    all_profiles.append({
        "did": data.get("did"),
        "handle": data.get("handle"),
        "displayName": data.get("displayName"),
        "followersCount": data.get("followersCount"),
        "followsCount": data.get("followsCount"),
        "postsCount": data.get("postsCount"),
        "createdAt": data.get("createdAt"),
        "description": data.get("description"),
    })

# Convert list of dictionaries into a DataFrame
all_profiles_df = pd.DataFrame(all_profiles)

# Display first few rows to verify
all_profiles_df.head(5)




Number of unique authors: 50


,did,handle,displayName,followersCount,followsCount,postsCount,createdAt,description
0,did:plc:thwspoidoksdkjdjclykbkzn,disderp.bsky.social,DisDerp,272,65,1517,2024-11-16T19:17:32.786Z,“LUCK BE IN THE AIR TONIGHT” 😂😂😂\n\nAll jokes ...
1,did:plc:5pzticukc5rgbmc7o3bso7il,mattsego.bsky.social,MSG,1364,944,11160,2023-06-22T13:36:45.516Z,"Who can never know, like, what will be told, a..."
2,did:plc:qixf4a5ossg2fkd3ztb3asyk,celinevale2.bsky.social,Celine Vale ❤️,233,378,133,2025-08-18T16:56:51.497Z,"Fantasy Illustrator 🎨 | DnD Art, Party Composi..."
3,did:plc:ijk2uijbwllxr2z7q3yyzily,ankemarsh.bsky.social,Dr Anke Marsh,4385,2861,2211,2023-11-07T07:39:19.516Z,"Palaeoecology, natural history, science, flora..."
4,did:plc:kjsl6b5t5klf5cbtnkyxq3ar,mel.bzky.team,meliss-AAAH!,15074,299,50077,2023-06-15T08:33:05.324Z,opinions ARE my employer's. retweets ARE endor...


## Section 4 - Build DataFrames

In [7]:
# Classic pandas stitch:
# merge joins rows from the two dataframes based on matching key values.
posts_enriched = posts_df.merge(
    # Adds "author_" to every column name in all_profiles_df
    # Why? To avoid name collisions (e.g., both dataframes could have handle, displayName) 
    # and to make the origin obvious: anything about the author now clearly starts with author_.
    all_profiles_df.add_prefix("author_"),
    # left_on="author_did": use posts_df["author_did"] as the join key on the left.
    left_on="author_did",
    # right_on="author_did": use the prefixed key from the right dataframe (formerly did).
    right_on="author_did",
    # how="left": a left join. Keep every row from posts_df (every post), 
    # even if there is no matching profile. If a profile is missing, 
    # the author columns become NaN. 
    # This is what you want for enrichment—don’t drop posts just because the profile lookup failed.
    how="left"
)

posts_enriched = posts_enriched.dropna(subset=["likeCount"])
posts_enriched = posts_enriched.sort_values(by="likeCount", ascending=False)
posts_enriched.head(5)


,post_uri,post_cid,author_did,likeCount,repostCount,has_alt_text,author_handle,author_displayName,author_followersCount,author_followsCount,author_postsCount,author_createdAt,author_description
49,at://did:plc:hdtepbxpmzsgrwz4gyiwjlar/app.bsky...,bafyreiew6txuvkupoaldb47pqzss2okrshxdijkwjqzwd...,did:plc:hdtepbxpmzsgrwz4gyiwjlar,13,2,True,jampupper.bsky.social,~ Jammy ~,2462,344,3718,2023-08-26T12:25:48.043Z,He/They | 33 | 🔞\nDancing coyutie pup excited ...
41,at://did:plc:uzlhbzc7w2s4np4lz3f7hazu/app.bsky...,bafyreid7f435rheet5grybqyt32v7ml6mu32ctqua4ub2...,did:plc:uzlhbzc7w2s4np4lz3f7hazu,12,0,True,amychu.bsky.social,☕️ AMY CHU #donutkiller,6067,987,739,2023-07-14T13:03:08.612Z,#Legohoarder #whiskeysipper \nI write comics g...
43,at://did:plc:end3y4t6sqefadytxixlgghe/app.bsky...,bafyreiet2p6wep56rtafagckpc2yango5hnm6fupuhcgj...,did:plc:end3y4t6sqefadytxixlgghe,7,0,True,angemmorton.bsky.social,Angela Marie Morton,3841,2093,2560,2023-09-21T04:15:49.325Z,○ art & illustration\n○ calm art for a busy wo...
47,at://did:plc:irec2mkigwl3zekio35covi2/app.bsky...,bafyreida32xj3hxnnhh3nbbz3sdm6p4mfyzjrf4poollx...,did:plc:irec2mkigwl3zekio35covi2,6,0,True,unclebeard1978.bsky.social,Dr. Uncle Feared 🦊 🌈,3961,3172,23178,2023-09-03T13:17:52.134Z,"Doctor, author, bear in North Yorkshire. Docto..."
29,at://did:plc:xiubhwhnsfqz2rfbmhhdnvns/app.bsky...,bafyreibdjsywd4kb6j5ljgt7sopfnjg7tgzhzej65mazq...,did:plc:xiubhwhnsfqz2rfbmhhdnvns,4,1,False,minisanctuary.bsky.social,Mini Motley Sanctuary,15721,57994,735,2024-11-09T11:48:55.679Z,A mother daughter team running our special san...


# Section 5 — Conclusion

First look of the dataset shows two groups of image posts—those that include alt text and ones that do not. Early analysis suggests that posts with alt text are usually shown to, and supported by users who care a lot about community and are woke, and they have higher engagement with posts including alt text.  

While the sample size is limited, the results show, supporting the hypothesis, that those who show that they are conscious with accessibility postings (alt text included) correlates with greater user engagement for those posts. 

**Challenges faced:**  
- Bluesky API rate limits restricted sample size.  
- Some posts had missing `embed` data.  
- Comparing in real time (current) is hard because they don't update right away.  

**Next steps:**  
Widening the dataset could further show and prove the patterns that are seen and represented, using a t-test. Future tests could check the quality of alt text, to test whether deeper and more detailed descriptions show higher engagement.

**AI Use: ChatGPT (free version)**
I used AI for help with formatting and pasted the outline from the Github to compare and it helped me fix a few minor formatting issues. There were a few brainstorming prompts for the hypothesis. I also had a few prompts written to double check that the template and my work matched up good, following the grading guidelines and matching the template, and also a few fixes for minor code errors I wasn't sure how to fix on my own. 